In [1]:
class Module:
    def __init__(self, receivers: list[str]):
        self.receivers = receivers
        self.pulse_counts = {'low':0, 'high':0}
    
    def init_conjunctions(self, name, modules):
        modules_out = dict()
        for r in self.receivers:
            if r not in modules.keys():
                modules_out[r] = Output()
            elif type(modules[r])==Conjunction:
                modules[r].memory[name] = 'low'
        return modules | modules_out

class FlipFlop(Module):
    def __init__(self, receivers):
        self.state='off'
        super().__init__(receivers)
    
    def pulse(self, pulse, module_name):
        if pulse=='low':
            if self.state=='off':
                self.state = 'on'
                return 'high'
            else:
                self.state = 'off'
                return 'low'
        else:
            return None
    
class Conjunction(Module):
    def __init__(self, receivers):
        self.memory = dict()
        super().__init__(receivers)

    def pulse(self, pulse, module_name):
        self.memory[module_name] = pulse

        if any([p=='high' for p in self.memory.values()]) and (module_name=='gq'):
            print("!")

        if all([p=='high' for p in self.memory.values()]):
            return 'low'
        else:
            return 'high'

class Broadcast(Module):
    def pulse(self, pulse, module_name):
        return pulse
    
class Button(Module):
    def pulse(self, *_):
        return 'low'
    
class Output(Module):
    def __init__(self):
        super().__init__([])
    
    def pulse(self, *_):
        return None
    
class Modules:
    def __init__(self, input, verbose=False):
        self.verbose = verbose
        self.modules = dict()
        for line in input.splitlines():
            module, receivers = line.split(" -> ")
            receivers = receivers.split(", ")

            match module[0]:
                case '%': self.modules[module[1:]] = FlipFlop(receivers)
                case '&': self.modules[module[1:]] = Conjunction(receivers)
                case 'b': self.modules[module] = Broadcast(receivers)
        
        self.modules['button'] = Button(['broadcaster'])
        self.modules['output'] = Output()
        self.init_conjunctions()

    def init_conjunctions(self):
        modules_out = dict()
        for name, module in self.modules.items():
            modules_out = modules_out | module.init_conjunctions(name, self.modules)
        self.modules = modules_out

    def pulse(self, senders: list[str], receivers: list[str], pulses_in: list[str]):
        modules_out = list()
        pulses_out = list()
        receivers_out = list()
         
        for sender_name, receiver_name, pulse_in in zip(senders, receivers, pulses_in): 
            module = self.modules[receiver_name]
            if pulse_out := module.pulse(pulse_in, sender_name):
                for next_receiver in module.receivers:
                    if self.verbose:
                        print(receiver_name, "->", pulse_out, "->", next_receiver)
                    receivers_out.append(receiver_name)
                    modules_out.append(next_receiver)
                    pulses_out.append(pulse_out)

                    module.pulse_counts[pulse_out] += 1
                
        return receivers_out, modules_out, pulses_out


    def get_results(self):
        low = sum([m.pulse_counts['low'] for m in self.modules.values()])
        high = sum([m.pulse_counts['high'] for m in self.modules.values()])
        return low*high
    
    def run(self, n=1):
        for _ in range(n):
            out = [''], ['button'], ['']
            while out[0]:
                out = self.pulse(*out)



In [2]:
with open("inputs/day20.txt", 'r') as fn:
    input = fn.read()

modules = Modules(input)
modules.run(1000)
modules.get_results()

832957356

# Part 2

In [3]:
[(k,v.receivers) for k, v in modules.modules.items() if 'rx' in v.receivers]

[('gq', ['rx'])]

In [4]:
type(modules.modules['gq'])

__main__.Conjunction

In [5]:
gq_inputs = [k for k, v in modules.modules.items() if 'gq' in v.receivers]
gq_inputs

['xj', 'qs', 'kz', 'km']

> We need each of the above to be 'high' to get a 'high' to gq

In [6]:
with open("inputs/day20.txt", 'r') as fn:
    input = fn.read()

modules = Modules(input)

gq_tracker = {k: list() for k in gq_inputs}
_gq_tracker = {k: 0 for k in gq_inputs}
for i in range(20_000):
    modules.run(1)
    for k in gq_inputs:
        if (_i:=modules.modules[k].pulse_counts['high']) > _gq_tracker[k]:
            gq_tracker[k].append(i+1)
            _gq_tracker[k] = _i

In [7]:
gq_tracker

{'xj': [3733, 7466, 11199, 14932, 18665],
 'qs': [4019, 8038, 12057, 16076],
 'kz': [3911, 7822, 11733, 15644, 19555],
 'km': [4093, 8186, 12279, 16372]}

In [8]:
from math import lcm
lcm(3733, 4019, 3911, 4093)

240162699605221